# NeoHealth AI – Menstrual Phase Prediction Model


## Objective
This notebook trains a machine learning model to predict the current menstrual cycle phase (Menstrual, Follicular, Ovulation, Luteal) using the processed dataset generated in the data preprocessing notebook.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier

In [2]:
df = pd.read_csv("../data/processed/final_dataset.csv")
print(df.shape)
df.head()

(1524, 31)


,id,day_in_study,phase,lh,estrogen,pdg,cramps,fatigue,moodswing,stress,...,estrogen_prev2,pdg_prev1,pdg_prev2,stress_prev1,stress_prev2,overall_score_prev1,overall_score_prev2,daily_steps_prev1,daily_steps_prev2,phase_simple
0,2,3,Fertility,0.030711,0.292969,0.181036,0.0,1.0,0.0,3.0,...,78.5,6.250051,6.250051,2.0,2.0,90.0,87.0,1584.0,1366.0,Peak Hormone
1,2,4,Fertility,0.142241,0.344219,0.181036,0.0,3.0,2.0,3.0,...,126.9,6.250051,6.250051,3.0,2.0,89.0,90.0,1189.0,1584.0,Peak Hormone
2,2,5,Fertility,0.105065,0.449531,0.181036,0.0,3.0,2.0,3.0,...,187.5,6.250051,6.250051,3.0,3.0,80.0,89.0,6654.0,1189.0,Peak Hormone
3,2,6,Fertility,0.024784,0.183281,0.181036,0.0,2.0,2.0,4.0,...,220.3,6.250051,6.250051,3.0,3.0,71.0,80.0,4760.0,6654.0,Peak Hormone
4,2,7,Luteal,0.023707,0.155938,0.181036,0.0,2.0,0.0,1.0,...,287.7,6.250051,6.250051,4.0,3.0,87.0,71.0,3818.0,4760.0,High Progesterone


In [3]:
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1524 entries, 0 to 1523
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      1524 non-null   int64  
 1   day_in_study            1524 non-null   int64  
 2   phase                   1524 non-null   object 
 3   lh                      1524 non-null   float64
 4   estrogen                1524 non-null   float64
 5   pdg                     1524 non-null   float64
 6   cramps                  1524 non-null   float64
 7   fatigue                 1524 non-null   float64
 8   moodswing               1524 non-null   float64
 9   stress                  1524 non-null   float64
 10  bloating                1524 non-null   float64
 11  sleepissue              1524 non-null   float64
 12  overall_score           1524 non-null   float64
 13  deep_sleep_in_minutes   1524 non-null   float64
 14  resting_heart_rate      1524 non-null   

id                        0
day_in_study              0
phase                     0
lh                        0
estrogen                  0
pdg                       0
cramps                    0
fatigue                   0
moodswing                 0
stress                    0
bloating                  0
sleepissue                0
overall_score             0
deep_sleep_in_minutes     0
resting_heart_rate        0
stress_score              0
avg_resting_heart_rate    0
daily_steps               0
lh_prev1                  0
lh_prev2                  0
estrogen_prev1            0
estrogen_prev2            0
pdg_prev1                 0
pdg_prev2                 0
stress_prev1              0
stress_prev2              0
overall_score_prev1       0
overall_score_prev2       0
daily_steps_prev1         0
daily_steps_prev2         0
phase_simple              0
dtype: int64

In [4]:
le = LabelEncoder()
df["phase_encoded"] = le.fit_transform(df["phase_simple"])  # use simplified phase

print(le.classes_)

['High Progesterone' 'Low Hormone' 'Peak Hormone' 'Rising Hormone']


In [5]:
features = [
    "lh","estrogen","pdg",
    "cramps","fatigue","moodswing",
    "stress","bloating","sleepissue",
    "overall_score","deep_sleep_in_minutes",
    "avg_resting_heart_rate","stress_score","daily_steps",
    "lh_prev1","lh_prev2","estrogen_prev1","pdg_prev1","stress_prev1"
]

X = df[features].astype(float)
y = df["phase_encoded"].astype(int)

print(X.shape, y.shape)

(1524, 19) (1524,)


In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [8]:
models = {
    "Random Forest": RandomForestClassifier(n_estimators=300, class_weight="balanced", random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "XGBoost": XGBClassifier(objective="multi:softmax", num_class=len(np.unique(y)), eval_metric="mlogloss"),
    "SVM": SVC(kernel="rbf"),
    "KNN": KNeighborsClassifier(n_neighbors=7)
}

In [ ]:
results = {}
reports = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    
    labels_in_test = np.unique(y_test)
    report = classification_report(
        y_test, y_pred,
        labels=labels_in_test,
        target_names=le.inverse_transform(labels_in_test),
        output_dict=True
    )
    reports[name] = report
    
    print(f"\n{name} Accuracy: {acc}")
    print(classification_report(
        y_test, y_pred,
        labels=labels_in_test,
        target_names=le.inverse_transform(labels_in_test)
    ))


Random Forest Accuracy: 0.5639344262295082
                   precision    recall  f1-score   support

High Progesterone       0.55      0.79      0.65       111
      Low Hormone       0.75      0.44      0.55        55
     Peak Hormone       0.62      0.46      0.53        71
   Rising Hormone       0.46      0.40      0.43        68

         accuracy                           0.56       305
        macro avg       0.59      0.52      0.54       305
     weighted avg       0.58      0.56      0.55       305


Gradient Boosting Accuracy: 0.5606557377049181
                   precision    recall  f1-score   support

High Progesterone       0.56      0.74      0.64       111
      Low Hormone       0.76      0.47      0.58        55
     Peak Hormone       0.60      0.51      0.55        71
   Rising Hormone       0.42      0.40      0.41        68

         accuracy                           0.56       305
        macro avg       0.59      0.53      0.54       305
     weighted avg 

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x=list(results.keys()), y=list(results.values()))
plt.xticks(rotation=45)
plt.ylabel("Accuracy")
plt.title("Model Accuracy Comparison")
plt.show()

In [ ]:
best_model_name = max(results, key=results.get)
best_model = models[best_model_name]

y_pred_best = best_model.predict(X_test)

cm = confusion_matrix(y_test, y_pred_best)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.title(f"Confusion Matrix - {best_model_name}")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

print("Best Model:", best_model_name)

In [ ]:
param_grid = {
    "n_estimators": [200,300],
    "max_depth": [4,6,8],
    "learning_rate": [0.05,0.1],
    "subsample": [0.8,1.0]
}

grid = GridSearchCV(
    XGBClassifier(objective="multi:softmax", num_class=len(np.unique(y)), eval_metric="mlogloss"),
    param_grid,
    cv=3,
    scoring="accuracy",
    n_jobs=-1
)

grid.fit(X_train, y_train)

best_xgb = grid.best_estimator_
print("Best Params:", grid.best_params_)

In [ ]:
y_pred_tuned = best_xgb.predict(X_test)
acc_tuned = accuracy_score(y_test, y_pred_tuned)

print("Tuned XGBoost Accuracy:", acc_tuned)

labels_in_test = np.unique(y_test)

print(classification_report(
    y_test, y_pred_tuned,
    labels=labels_in_test,
    target_names=le.inverse_transform(labels_in_test)
))

In [ ]:
importances = best_xgb.feature_importances_

feat_df = pd.DataFrame({
    "Feature": features,
    "Importance": importances
}).sort_values(by="Importance", ascending=False)

plt.figure(figsize=(8,6))
sns.barplot(x="Importance", y="Feature", data=feat_df.head(15))
plt.title("Top 15 Feature Importances")
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x=df["phase_simple"])
plt.title("Phase Distribution")
plt.xticks(rotation=30)
plt.show()

In [ ]:
all_acc = list(results.values()) + [acc_tuned]
labels = list(results.keys()) + ["Tuned XGBoost"]

plt.figure(figsize=(10,6))
sns.barplot(x=labels, y=all_acc)
plt.xticks(rotation=45)
plt.ylabel("Accuracy")
plt.title("Accuracy Comparison (All Models)")
plt.show()

In [ ]:
best_xgb = grid_xgb.best_estimator_

y_pred_xgb = best_xgb.predict(X_test)

acc_xgb = accuracy_score(y_test, y_pred_xgb)
print("XGBoost Accuracy:", acc_xgb)

labels_in_test = np.unique(y_test)

print(classification_report(
    y_test,
    y_pred_xgb,
    labels=labels_in_test,
    target_names=le.inverse_transform(labels_in_test)
))

In [ ]:
estimators = [
    ("rf", rf_model),
    ("gb", gb_model),
    ("xgb", best_xgb)
]

stack_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(max_iter=1000),
    cv=3
)

stack_model.fit(X_train, y_train)

y_pred_stack = stack_model.predict(X_test)

acc_stack = accuracy_score(y_test, y_pred_stack)
print("Stacking Model Accuracy:", acc_stack)

labels_in_test = np.unique(y_test)

print(classification_report(
    y_test,
    y_pred_stack,
    labels=labels_in_test,
    target_names=le.inverse_transform(labels_in_test)
))

In [ ]:
cv_scores = cross_val_score(best_xgb, X_scaled, y, cv=5, scoring="accuracy")

print("Cross-validation scores:", cv_scores)
print("Mean CV accuracy:", cv_scores.mean())

In [ ]:
class_counts = pd.Series(y).value_counts()
print(class_counts)

In [ ]:
min_samples = 10

valid_classes = class_counts[class_counts >= min_samples].index

mask = y.isin(valid_classes)

X_filtered = X_scaled[mask]
y_filtered = y[mask]

print("New class distribution:")
print(pd.Series(y_filtered).value_counts())

In [ ]:
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(
    X_filtered,
    y_filtered,
    test_size=0.2,
    random_state=42,
    stratify=y_filtered
)

In [ ]:

xgb_adv = XGBClassifier(
    n_estimators=400,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    objective="multi:softmax",
    num_class=len(np.unique(y_filtered)),
    eval_metric="mlogloss",
    random_state=42
)

xgb_adv.fit(X_train_f, y_train_f)

y_pred_adv = xgb_adv.predict(X_test_f)

acc_adv = accuracy_score(y_test_f, y_pred_adv)
print("Advanced XGBoost Accuracy:", acc_adv)

labels_in_test = np.unique(y_test_f)

print(classification_report(
    y_test_f,
    y_pred_adv,
    labels=labels_in_test,
    target_names=le.inverse_transform(labels_in_test)
))

In [ ]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

cv_scores = cross_val_score(
    xgb_adv,
    X_filtered,
    y_filtered,
    cv=skf,
    scoring="accuracy"
)

print("Stratified CV scores:", cv_scores)
print("Mean CV accuracy:", cv_scores.mean())
